# Lecture 3 - Manipulating Data Frames and Calculating Summaries

## Today's Key Takeaways

- **Reshaping** data frames
- Calculating **summary parameters**
- **Sorting** data in `pandas`

### Adding libraries to python

First we need to import the libraries that we'll want to use for today's class. This is the same process you used for the last homework to add a function to python, but libraries can add hundreds of new functions.

When we import these libraries we can assign them an alias, which is easier to remember and type. The ones used below are common for these packages. 

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd


### From Last Time:

Start by importing the metadata (same as last week).

In [ ]:
# Alternative way to bring in Data
from google.colab import files
uploaded = files.upload()


In [ ]:
# Bring in metadata
CountsData = pd.read_csv("melanoma_CountsRaw.csv", index_col=0)
CountsData.head()


### Subsetting

One of the powerful features of data frames is that rows and columns can be referred to by names or numbers:

In [ ]:
CountsData['Stage']

To select multiple rows or columns, input a list:

In [ ]:
CountsData.loc[['FM_1','FM_2','FM_3'],'Stage']

You can also subset data by number by using "`.iloc`" (zero-indexing also applies here)

In [ ]:
CountsData.iloc[0:3,2]

Data frames can also be sliced by rows without explicitly naming the row names or row numbers. Instead, we can slice by testing whether a condition is fulfilled or not:

In [ ]:
# Finding out which rows match the stage: "primary melanocytes"
primary_indices = CountsData.Stage == 'primary melanocytes'
print(primary_indices)

In [ ]:
# subsetting the rows based on this conditional set
primary_melanocytes_data = CountsData[primary_indices]
primary_melanocytes_data

### <font color=blue> Optional Bonus: Data Manipulation - adding new columns to data frames 

We can add a new column to a data frame using the same syntax as assigning a column slice to a data structure that contains the same number of elements as rows in the data frame. 

In [ ]:
# Find out the number of rows and columns with the .shape attribute
CountsData.shape

`CountsData` has 12 rows and 35,243 columns. So say we want to add an extra column to `CountsData` that includes the replicate number for each sample (i.e. either 1, 2, or 3).

There are multiple ways of doing this, but one of the ways would be to create a list that looks like [1,2,3,1,2,...].

We can do this by first creating a `numpy` array that contains the numbers 1-3, convert it to a list, then use list operations to duplicate it 3 more times. We can then assign this to a new column in the `CountsData` data frame, that we'll call `replicate_number`.

In [ ]:
# Creating a numpy array of that starts from 1 and ends before 4, stepping by 1, using the np.arange() function.
tmp =  np.arange(1,4)
print(tmp)



In [ ]:
# Converting to a list
tmp = tmp.tolist()

# Duplicating the list three additional times
tmp = tmp * 4
print(tmp)

In [ ]:
# Assigning a new column called 'sample_number' in the data frame meta with the values in tmp
CountsData['replicate_number'] = tmp
CountsData.head()

We can rearrange the column order so that the replicate number is more visible during `.head()` calls. Other examples of how to do this can be found [here](https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns) and [here](https://stackoverflow.com/questions/53141240/pandas-how-to-swap-or-reorder-columns).

In [ ]:
cols = CountsData.columns.tolist() # Extract the column names and save it as a list.
print(cols)
print(len(cols)) # Sanity check: print out number of columns

In [ ]:
colsRearranged = cols[:3] + cols[-1:] + cols[3:-1] # Use list operations to rearrange the order of the columns
print(colsRearranged)
print(len(colsRearranged)) # Sanity check: print out number of rearranged columns

In [ ]:
CountsData = CountsData[colsRearranged] # Slice the data frame by the rearranged column order, and then save it as the rearranged data frame
CountsData.head()

## Data Manipulation - removing columns with low counts by conditional slicing

Often times when working with high-throughput experimental data, there will be specific columns (corresponding to variables) that we won't want to consider, because those columns contain data that don't pass a particular inclusion criteria. This case study (removing columns with low counts), will show how to remove the columns that don't pass the inclusion criteria from the data frame without having to know beforehand which column numbers they are. 

To start, we'll first create a variable that only stores the subset of the data frame that contains the expression count data.

In [ ]:
dataExp = CountsData.loc[:,'A1BG':]
dataExp.head()

In [ ]:
# Make a Data Frame of boolean values showing where there are read counts less than 1
df_low = dataExp < 1
df_low

In [ ]:
# Add together all of the columns to get total number of samples with low counts
lowsum = df_low.sum()
lowsum.head()

In [ ]:
# Showing which samples have low expression of the gene A1CF
df_low.A1CF

In [ ]:
# Subsetting the expression matrix on the samples deemed to be low in A1CF expression.
dataExp.loc[df_low.A1CF,'A1CF']

To help us get a sense of the distribution of the `lowsum` variable, we can use `pandas` calculation of summary parameters using the `.describe()` method. 

In [ ]:
# Looking at the quantiles
lowsum.describe()

You can see that many genes either have reads for all 12 samples or none of them. We want to remove all of those genes from our data frame. Let's do that by selecting on a list of genes that have reads for all 12 samples.

In [ ]:
# Make a mask of all the columns we want to keep
# Let's be picky and only accept genes where all 12 samples have counts
keep = lowsum < 1
keep.head()

In [ ]:
# Calculating how many are have reads for all 12 samples
np.sum(keep)

In [ ]:
# Using df.loc we can select just the genes we want to keep
Exp_low_removed = dataExp.loc[:,keep]

In [ ]:
# Check the shape before and after
print(dataExp.shape)
print(Exp_low_removed.shape)

In [ ]:
Exp_low_removed.head()

## <font color=brown>Hands on practice</font>
How many genes have less than 12 samples with reads, but more than zero?

### Data Manipulation: log2-transform expression counts

Another useful manipulation of data is to apply a mathematical operation uniformly across all of the values in a dataset. In this case study, we're going to apply the log2 transformation.

In [ ]:
# Applying the log2 transformation
Exp_low_removed_log2 = np.log2(Exp_low_removed)
Exp_low_removed_log2.head()

### Data Manipulation: Joining data frames

Let's join the `meta` data frame with the `df` data frame. To do this, we will use the `pandas` function `pd.concat()`. Note that this works similarly to `.hstack()` and `.vstack()` from numpy, but you must specify which axis to concatenate (0 = stitch by rows, equivalent to `vstack`, 1 = stitch by columns, equivalent to `hstack`). 

In [ ]:
meta = CountsData.loc[:,:'cell_line']
meta.head()

In [ ]:
# Merge the two data frames with concat
melanoma_log2 = pd.concat([meta,Exp_low_removed_log2],axis = 1)

In [ ]:
# Check the dimensions and show the first few lines of the new data frame
melanoma_log2.shape
melanoma_log2.head()

Let's export this data frame to excel, so that we can directly import this data frame in future classes.

In [ ]:
# export to excel, including the index column
melanoma_log2.to_excel('melanoma_zerosRemoved_log2transformed_2023.xlsx',index = True)

## Calculating Summary Parameters

`pandas` also enables calculation of summary parameters using the `.describe()` method (reference [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)). Note that this will only work effectively for small data frames, so in this example, we are only looking at the first 10 genes of `df_low_removed_log2` (we're using `df_low_removed_log2` instead of `melanoma_log2` because it doesn't have the metadata columns): 

In [ ]:
melanoma_log2.loc[:,'A1BG':'AACS'].describe()

You can also customize what your summary statistics will consist of with the `.aggregate()` method (reference [here](https://www.geeksforgeeks.org/python-pandas-dataframe-aggregate/) and [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html)). 

In [ ]:
# Calculating the minimum, median, max, and sum across rows of each column. 
# Note that the median function is found in numpy.
melanoma_log2.loc[:,'A1BG':'AACS'].aggregate([min,np.median,max,sum])

The set of aggregate metrics that can be calculated are:

- `mean` - the average
- `median` - the median
- `prod` - the product
- `sum` - the sum
- `std` - the standard deviation
- `var` - the variance

### Calculating overall metrics and sorting

For each of the metrics that you can use with `.aggregate()` (e.g. `df.mean()`, `df.min()`, etc.), you can calculate them individually.  

This is particularly useful when you want to find genes that are doing interesting things in your data.

In [ ]:
# Calculate the overall variance with df.var() 
overall_variance = melanoma_log2.loc[:,'A1BG':].var()
print(overall_variance.head())
print(overall_variance.shape)

**Case study**: we want to find genes that are highly variable across conditions, but we don't know which they are. Therefore, we're going to sort the overall variance values that we just calculated in descending order (i.e. from largest to smallest) using the `.sort_values()` method. Then, we'll be able to pick out which genes are the most variable

In [ ]:
# Sort this Series and select the 20 genes with the largest variance
overall_variance.sort_values(inplace = True, ascending= False)
overall_variance.head()

Notice that the `overall_variance` variable has changed, even though we haven't reassigned it (with the = sign). This is because we set the `.sort_values()` argument `inplace = True`. 

In [ ]:
# Now print the top 10 most variable genes.
topvar = overall_variance[:10]
print(topvar)
type(topvar)

We can pull out the gene information by accessing the `index` attribute of this `pandas series`:

In [ ]:
# We want to use the gene names, not their variance, to filter the columns of melanoma_log2
topvar.index

We can then use these indexes to slice the `melanoma_df` data frame and look at what the actual expression count values were for these genes.

In [ ]:
# Now we can use that Series with melanoma_log.loc to make a subtable
topvartable = melanoma_log2.loc[:,topvar.index]
topvartable

## Computation based on sample groups 

Our expression dataset has three replicates for each cell line. `pandas` enables calculation of multiple paramaters with these replicates consolidated, if your input data frame contains a variable that contains the grouping information. 

The replicates can be grouped together with the syntax:
```python 
df.groupby(['column_name'])
```

In [ ]:
# reminder of what melanoma_log2 looks like
melanoma_log2.head()

In [ ]:
# using .groupby() to designate groupings by cell line
mel_by_cel = melanoma_log2.groupby('cell_line')
mel_by_cel

We have made a "`DataFrameGroupBy`" object.

This is an iterator, an object that iterates over a function, offering it one block of data at a time. To generate the mean of each gene for each cell line, we use the following:

In [ ]:
# calculates group-by mean
mel_by_cel_mean = mel_by_cel.mean()
mel_by_cel_mean

In [ ]:
# calculates group-by variance
mel_by_cel_var = mel_by_cel.var()
mel_by_cel_var

 We can similarly calculate the `median`, `min`, `max`, `var`, and a host of other metrics.

### <font color=brown>Hands on practice</font>
1. Use `groupby` to calculate the mean expression counts across each stage.
2. Use `describe` to calculate the summary parameters of `mean_bystage` for the metastatic and primary melanocytes stages separately.
3. Find the top 10 variably expressed gene between the stages from the `mean_bystage` variable.
4. Display the `mean_bystage` table sliced to show only these top 10 genes.

For more information about grouping, summarization, and aggregation, [go here](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html).